In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import functions

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 
                'member_information.csv', 
                'labresults_training.csv', 
                'medical_training.csv', 
                'rx_training.csv',
                'medical_target.csv',
                'rx_target.csv']

with open('members.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open('trainingmedicalwanted.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

medical_wanted = pd.DataFrame(data)
medical_wanted.columns = medical_wanted.iloc[0]
medical_wanted = medical_wanted[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

with open('targetmedicalwanted.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

medical_wanted_t = pd.DataFrame(data)
medical_wanted_t.columns = medical_wanted_t.iloc[0]
medical_wanted_t = medical_wanted_t[1:]

with open(filepath+csvfilenames[6]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetrxdata = pd.DataFrame(data)
targetrxdata.columns = targetrxdata.iloc[0]
targetrxdata = targetrxdata[1:]

In [ ]:
#medical_wanted['STD_COST']=medical_wanted['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
members['year_of_birth']=members['year_of_birth'].apply(pd.to_numeric)
members['totalcost']=members['totalcost'].apply(pd.to_numeric)
members['targetcost']=members['targetcost'].apply(pd.to_numeric)
members['rxcost']=members['rxcost'].apply(pd.to_numeric)
members['targetrxcost']=members['targetrxcost'].apply(pd.to_numeric)
members['medicalcost']=members['medicalcost'].apply(pd.to_numeric)
members['targetmedicalcost']=members['targetmedicalcost'].apply(pd.to_numeric)
members['hospicevisits']=members['hospicevisits'].apply(pd.to_numeric)
members['officevisits']=members['officevisits'].apply(pd.to_numeric)
members['psychvisits']=members['psychvisits'].apply(pd.to_numeric)
members['emervisits']=members['emervisits'].apply(pd.to_numeric)
members['elderlyvisits']=members['elderlyvisits'].apply(pd.to_numeric)
members['unknownvisits']=members['unknownvisits'].apply(pd.to_numeric)
members['homevisits']=members['homevisits'].apply(pd.to_numeric)
members['dialysisvisits']=members['dialysisvisits'].apply(pd.to_numeric)

rxdata['DAYS_FROM_DIAG'] = rxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
#rxdata['QUANTITY']=rxdata['QUANTITY'].apply(pd.to_numeric) #unecessary for now, but potentially useful
medical_wanted['DAYS_FROM_DIAG'] = medical_wanted['DAYS_FROM_DIAG'].apply(pd.to_numeric)

#medical_wanted_t['STD_COST']=medical_wanted_t['STD_COST'].apply(pd.to_numeric) #change entries to numbers
#targetrxdata.loc[targetrxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
#targetrxdata['STD_COST']=targetrxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
#targetrxdata['DAYS_FROM_DIAG'] = targetrxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
#medical_wanted_t['DAYS_FROM_DIAG'] = medical_wanted_t['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
members['infpardisdiag']=0 #001-139
members['neoplasmsdiag']=0 #140-239
#members['endocrinediag']=1 #240-279, needs to be split up
members['blooddiag']=0 #280-289
members['mentaldiag']=0 #290-319
members['nervediag']=0 #320-359
members['sensediag']=0 #360-389
members['circdiag']=0 #390-489
members['respdiag']=0 #460-519
members['digestdiag']=0 #520-579
members['genitdiag']=0 #580-629
members['skindiag']=0 #608-709
members['muscskeldiag']=0 #710-739
members['conganomdiag']=0 #740-759
members['illdefdiag']=0 #780-799

dummy = medical_wanted.groupby('PATID')
patids = list(members.index)
for k in patids:
    try:
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['DIAG1']>='001') & (tinydummy['DIAG1']<='13999')]):
            members.loc[k,'infpardisdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='140') & (tinydummy['DIAG1']<='23999')]):
            members.loc[k,'neoplasmsdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='280') & (tinydummy['DIAG1']<='28999')]):
            members.loc[k,'blooddiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='290') & (tinydummy['DIAG1']<='31999')]):
            members.loc[k,'mentaldiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='320') & (tinydummy['DIAG1']<='35999')]):
            members.loc[k,'nervediag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='360') & (tinydummy['DIAG1']<='38999')]):
            members.loc[k,'sensediag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='390') & (tinydummy['DIAG1']<='48999')]):
            members.loc[k,'circdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='460') & (tinydummy['DIAG1']<='51999')]):
            members.loc[k,'respdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='520') & (tinydummy['DIAG1']<='57999')]):
            members.loc[k,'digestdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='580') & (tinydummy['DIAG1']<='62999')]):
            members.loc[k,'genitdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='608') & (tinydummy['DIAG1']<='70999')]):
            members.loc[k,'skindiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='710') & (tinydummy['DIAG1']<='73999')]):
            members.loc[k,'muscskeldiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='740') & (tinydummy['DIAG1']<='75999')]):
            members.loc[k,'conganomdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='780') & (tinydummy['DIAG1']<='79999')]):
            members.loc[k,'illdefdiag']=1  
    except KeyError:
        pass

In [ ]:
members['allergies'] =0 # 04
members['antiinfectives']=0 # 08
members['antineoplastic']=0 #10
members['autonomic']=0 #12
members['bloodfromcoag']=0 #20
members['cardiovascular']=0 #24
members['centralnervous']=0 #28
members['contraceptives']=0 #32
members['dental'] = 0 #34
members['electrolytic']=0 #40
members['enzymes']=0 #44
members['coughstuff']=0 #48
members['opthamlic']=0 #52
members['gastrointestinal']=0 #56
members['hormones']=0 #68
members['oxytocics']=0 #76
members['skinstuff']=0 #84
members['vitamins']=0 #88
members['asthma/alcohol'] = 0 #92

dummy = rxdata.groupby('PATID')
patids = list(members.index)
for k in patids:
    try:
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('04'))]):
            members.loc[k,'allergies']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('08'))]):
            members.loc[k,'antiinfectives']=0
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('10'))]):
            members.loc[k,'antineoplastic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('12'))]):
            members.loc[k,'autonomic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('20'))]):
            members.loc[k,'bloodfromcoag']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('24'))]):
            members.loc[k,'cardiovascular']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('28'))]):
            members.loc[k,'centralnervous']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('32'))]):
            members.loc[k,'contraceptives']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('34'))]):
            members.loc[k,'dental']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('40'))]):
            members.loc[k,'electrolytic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('44'))]):
            members.loc[k,'enzymes']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('48'))]):
            members.loc[k,'coughstuff']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('52'))]):
            members.loc[k,'opthamlic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('56'))]):
            members.loc[k,'gastrointestinal']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('68'))]):
            members.loc[k,'hormones']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('76'))]):
            members.loc[k,'oxytocics']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('84'))]):
            members.loc[k,'skinstuff']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('88'))]):
            members.loc[k,'vitamins']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('92'))]):
            members.loc[k,'asthma/alcohol']=1
    except KeyError:
        pass

In [ ]:
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minrxday'] = dummy.iat[k]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxrxday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minmedicalday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxmedicalday'] = dummy.iat[k]

In [ ]:
members['medicalvisits']=0
dummy = medical_wanted.groupby('PATID')
patids = list(members.index)

for k in patids:
    try:
        members.loc[k,'medicalvisits']=len(dummy.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass

In [ ]:
members['rxfills']=0
dummy = rxdata.groupby('PATID')
patids = list(members.index)

for k in patids:
    try:
        members.loc[k,'rxfills']=len(dummy.get_group(k))
    except KeyError:
        pass

In [ ]:
members['rxperdoc']=members['rxfills']/(members['medicalvisits']+1) #to avoid infinity

In [ ]:
#incredibly slow, probs don't bother running this.

members['avgdaysrxnodoctor']=0
dummy=rxdata.groupby('PATID')
dummy2=medical_wanted.groupby('PATID')
patids = list(members.index)


for k in patids:
    try:
        mylist = []
        rxvisits = list(dummy.get_group(k)['DAYS_FROM_DIAG'])
        for j in rxvisits:
            mylist.append(np.absolute(dummy2.get_group(k)['DAYS_FROM_DIAG']-j).min())           
            members.loc[k, 'avgdaysrxnodoctor']=np.mean(mylist)
    except KeyError:
        pass
    

In [ ]:
tinydummy = dummy.get_group(k)['DAYS_FROM_DIAG'])
        tinydummy['daysfromdoctor']=0

In [ ]:
#Interesting codes in AHFSCLSS (so far) 
#681200: birthcontrol, 
#682004: biguanides (diabetes related), turns out to be useless
#682008: insulin 
#281604: antidepressents
#241212: ED drugs
#682020: Glypizide class (some kind of diabetes thing), useless
#28080424: Aspirins (salicilate), useless

In [ ]:
members['daysperinsulinrx']=0
members['daysperantideprx']=0
members['daysperbcrx']=0
members['daysperedrx']=0
members['daysperglyprx']=0
members['daysperasprinrx']=0

patids = list(members.index)
rxdata = rxdata.sort_values('DAYS_FROM_DIAG')
dummy = rxdata.groupby('PATID')
for k in patids:
    try:
        tinydummy=dummy.get_group(k)
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('682008')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperinsulinrx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('281604')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperantideprx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('681200')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperbcrx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('241212')]
        if len(moretinydummy)>0:
            members.loc[k,'daysperedrx']=moretinydummy['DAYS_FROM_DIAG'].diff(periods=1).mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('682020')]


    except KeyError:
        pass

In [ ]:
members['avginsulincost']=0
members['avgantidepcost']=0
members['averagebccost']=0

patids = list(members.index)
rxdata = rxdata.sort_values('DAYS_FROM_DIAG')
dummy = rxdata.groupby('PATID')
for k in patids:
    try:
        tinydummy=dummy.get_group(k)
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('682008')]
        if len(moretinydummy)>0:
            members.loc[k,'avginsulincost']=moretinydummy['STD_COST'].mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('281604')]
        if len(moretinydummy)>0:
            members.loc[k,'avgantidepcost']=moretinydummy['STD_COST'].mean()
        moretinydummy=tinydummy[tinydummy['AHFSCLSS'].str.startswith('681200')]
        if len(moretinydummy)>0:
            members.loc[k,'averagebccost']=moretinydummy['DAYS_FROM_DIAG'].mean()
    except KeyError:
        pass

In [ ]:
members['lasttimerxnodoctor']=np.absolute(members['maxrxday']-members['maxmedicalday'])
members['firsttimerxnodoctor']=np.absolute(members['minmedicalday']-members['minrxday'])

In [ ]:
#For making plots make more sense

#msk = np.random.rand(len(members)) < 0.02
#submembers = members[msk]

In [ ]:
#Our one good plot

#=most recent rx - most recent medical visit + least recent office visit - least recent rx
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='lasttimerxnodoctor', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) #not upper limit for this subpopulation
#plt.xlim(-50, 50)
plt.show()

In [ ]:
#next cells generate dummy variables for categorical; drop variables believed to be be unwanted.

In [ ]:
members_for_work = members
dummies = pd.get_dummies(members_for_work.region)
members_for_work = members_for_work.join(dummies)
dummies = pd.get_dummies(members_for_work.gender)
members_for_work = members_for_work.join(dummies)

In [ ]:
members_for_work = members_for_work.drop('gender', axis = 1)
members_for_work = members_for_work.drop('STATE', axis = 1)
members_for_work = members_for_work.drop('region', axis = 1)
#members_for_work = members_for_work.drop('minmedicalday', axis = 1)
#members_for_work = members_for_work.drop('maxmedicalday', axis = 1)
#members_for_work = members_for_work.drop('minrxday', axis = 1)
#members_for_work = members_for_work.drop('maxrxday', axis = 1)
members_for_work = members_for_work.drop('targetrxcost', axis = 1)
members_for_work = members_for_work.drop('targetmedicalcost', axis = 1)
members_for_work=members_for_work.fillna(0)

In [ ]:
#the next few cells run a really lousy linear regression.  At least if you're lucky they do.
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
members_train = members_for_work[:30000]
members_test = members_for_work[30000:]
members_y_train = members_train[['targetcost']]
members_x_train = members_train.drop('targetcost', axis=1)
members_y_test = members_test[['targetcost']]
members_x_test = members_test.drop('targetcost', axis=1)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(members_x_train, members_y_train)
members_y_pred= (regr.predict(members_x_test))

In [ ]:
mean_squared_error(members_y_test, members_y_pred)

In [ ]:
regr.coef_

In [ ]:
members_x_train.columns

In [ ]:
plt.scatter(members_y_test, members_y_pred)
plt.xlim=(0,100000)
plt.show()

In [ ]:
#These cells run various tree algorithms.  Whee

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
members_for_trees = members_for_work

In [ ]:
#Drop variables known to be useless for both types of trees (see end of notebook for code to checkvariable strength)
members_for_trees = members_for_trees.drop('psychvisits', axis = 1)
members_for_trees = members_for_trees.drop('elderlyvisits', axis = 1)
members_for_trees = members_for_trees.drop('dialysisvisits', axis = 1)
members_for_trees = members_for_trees.drop('hospicevisits', axis = 1)

In [ ]:
#Make True/False variable for our trees
members_for_trees_expensive = members_for_trees[members_for_trees['targetcost']>=12500]
members_for_trees_expensive['expensive']=True
members_for_trees_inexpensive = members_for_trees[members_for_trees['targetcost']<12500]
members_for_trees_inexpensive['expensive']=False
members_for_trees = members_for_trees_expensive.append(members_for_trees_inexpensive)

members_for_trees_vexpensive = members_for_trees[members_for_trees['targetcost']>=100000]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

#Make a little dataframe that's half expensive, half not, choosing a random collection on inexpensive each time
percentage = float(len(members_for_trees_vexpensive))/len(members_for_trees_inexpensive)
msk = np.random.rand(len(members_for_trees_inexpensive)) < percentage
members_for_small_trees = members_for_trees_expensive.append(members_for_trees_inexpensive[msk])

print len(members_for_trees_inexpensive[msk])
#Now choose a random subset of that for training/testing
msk = np.random.rand(len(members_for_small_trees)) < 0.8
members_small_train = members_for_small_trees[msk]
members_small_test = members_for_small_trees[~msk]

#Take away the answers
y_train = members_small_train['expensive']
y_test = members_small_test['expensive']
x_train = members_small_train.drop('expensive', axis = 1)
msk = np.random.rand(len(members_for_trees)) < 0.8

x_train = x_train.drop('targetcost', axis = 1)
x_test = members_small_test.drop('expensive', axis = 1)
x_test = x_test.drop('targetcost', axis = 1)

#Make the tree
dt = DecisionTreeClassifier(min_samples_split=200)
dt.fit(x_train, y_train)

#Add predictions to original stuff
pred = list(dt.predict(x_test))
members_small_test['pred']=pred
#hell if I know how to make a confusion matrix
check=members_small_test[['targetcost','expensive', 'pred']].sort_values('targetcost', ascending = 0)
check_truetrue = check[check['expensive'] & check['pred']]
check_true = check[check['expensive']]
check_predtrue = check[check['pred']]
#check_falsefalse = check[(not check['expensive']) & (not check['pred'])]
#check_falseneg = check[(check['expensive']) & (not check['pred'])]
#check_falsepos = check[(not check['expensive']) & (check['pred'])]
print float(len(check_truetrue))/len(check_true)
print float(len(check_truetrue))/len(check_predtrue)

In [ ]:
#All the same stuff, but with the full data.  Run as is to test your tree from above on the full data. (SPOILER ALERT: it sucks)
#remove comments on dt below to train a tree on the full data set.
msk = np.random.rand(len(members_for_trees)) < 0.8
members_train = members_for_trees[msk]
members_test = members_for_trees[~msk]
print len(members_train)
print len(members_test)

y_train = members_train['expensive']
y_test = members_test['expensive']
x_train = members_train.drop('expensive', axis = 1)
x_train = x_train.drop('targetcost', axis = 1)
x_test = members_test.drop('expensive', axis = 1)
x_test = x_test.drop('targetcost', axis = 1)

#dt = DecisionTreeClassifier(min_samples_split=500)
#dt.fit(x_train, y_train)

#I played around with samples_split and we're not going to do better than 33% true accuracy, it would seem
pred = list(dt.predict(x_test))
members_test['pred']=pred
#hell if I know how to make a confusion matrix
check=members_test[['targetcost','expensive', 'pred']].sort_values('targetcost', ascending = 0)
check_truetrue = check[check['expensive'] & check['pred']]
check_true = check[check['expensive']]
check_predtrue = check[check['pred']]
#check_falsefalse = check[(not check['expensive']) & (not check['pred'])]
#check_falseneg = check[(check['expensive']) & (not check['pred'])]
#check_falsepos = check[(not check['expensive']) & (check['pred'])]
print float(len(check_truetrue))/len(check_true)
print float(len(check_truetrue))/len(check_predtrue)

In [ ]:
#crossvalidation setup - presently designed for big trees
y_train = members_for_trees['expensive']
x_train = members_for_trees.drop('expensive', axis = 1)
x_train = x_train.drop('targetcost', axis = 1)

from sklearn.metrics import precision_score, make_scorer
from sklearn.model_selection import cross_val_score
score = make_scorer(precision_score)

output = []
for k in range(10):
    dt = DecisionTreeClassifier(min_samples_split=100*(k+1))
    scores = cross_val_score(dt, x_train, y_train, cv=10, scoring = score)
    output.append(sum(scores)/len(scores))

In [ ]:
output

In [ ]:
#I copy pasted this from stackoverflow
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print "def tree({}):".format(", ".join(feature_names))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print "{}if {} <= {}:".format(indent, name, threshold)
            recurse(tree_.children_left[node], depth + 1)
            print "{}else:  # if {} > {}".format(indent, name, threshold)
            recurse(tree_.children_right[node], depth + 1)
        else:
            print "{}return {}".format(indent, tree_.value[node])

    recurse(0, 1)

In [ ]:
#It generates some cool stuff like so:
tree_to_code(dt, x_train.columns)

In [ ]:
#Crossvalidation for variable importance for trees on little sets.

importances = [0 for k in range(71)]
k=0
while k < 100:
    percentage = float(len(members_for_trees_expensive))/len(members_for_trees_inexpensive)
    msk = np.random.rand(len(members_for_trees_inexpensive)) < percentage
    members_for_small_trees = members_for_trees_expensive.append(members_for_trees_inexpensive[msk])
    y_train = members_for_small_trees['expensive']
    x_train = members_for_small_trees.drop('expensive', axis = 1)
    x_train = x_train.drop('targetcost', axis = 1)

    from sklearn.ensemble import ExtraTreesClassifier
    model = ExtraTreesClassifier()
    model.fit(x_train, y_train)
    importances += model.feature_importances_

    k += 1
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_train.shape[1]), importances,
        color="r", align="center")
plt.xticks(range(x_train.shape[1]), range(len(x_train.columns)))
plt.xlim([-1, x_train.shape[1]])
plt.show()

In [ ]:
#happy counting

x_train.columns 